In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

import warnings # 경고 메세지 무시
import logging # 로그 생성
from warnings import simplefilter

In [ ]:
# tensorflow device 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2024-01-08 14:51:29.574757: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 14:51:29.574800: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 14:51:29.574813: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 14:51:29.579035: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 14:51:30.486512: I tensorflow/compiler/

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13519678852191013092
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7917797376
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1356285894059764148
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:05:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

2024-01-08 14:51:30.489126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-08 14:51:30.489168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-08 14:51:30.582463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-08 14:51:30.582605: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-08 14:51:30.582618: I tensorflow/core/co

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.list_physical_devices('GPU') 이 코드도 가능
for gpu in gpus:
    print(gpu)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-01-08 14:51:31.568002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-08 14:51:31.568070: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-08 14:51:31.568099: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-08 14:51:31.568257: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-08 14:51:31.568269: I tensorflow/core/co

In [ ]:
df_train = pd.read_csv('./train.csv')
df_test= pd.read_csv('./test.csv')
test = pd.read_csv('./sample_submission.csv')

In [ ]:
df_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
label = df_train['label']
pix = df_train.copy()
pix.drop(['label'], axis = 1, inplace = True)

In [ ]:
pix

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(pix, label, test_size=0.2, random_state=42)

In [ ]:
X_train_li = X_train.to_numpy()

for i in range(len(X_train_li)):
    X_train_li[i] = np.resize(X_train_li[i], (28,28))

print(X_train_li.shape)

'''
for i in range(len(X_train_li)):
    X_train_li[i] = X_train_li[i].reshape(28,28)

print(X_train_li.shape)
'''

ValueError: could not broadcast input array from shape (28,28) into shape (784,)

In [ ]:
#Image Classifier
import autokeras as ak

clf = ak.ImageClassifier(overwrite=True, max_trials=1)
clf.fit(a, y_train, epochs=10)

Using TensorFlow backend


NameError: name 'a' is not defined

In [ ]:
# h2o
import h2o
from h2o.sklearn import H2OAutoMLClassifier
warnings.filterwarnings('ignore')
logging.captureWarnings(True)
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
h2o.init()
aml = H2OAutoMLClassifier(max_runtime_secs = 180)
aml.fit(X_train,y_train.values)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_392"; OpenJDK Runtime Environment (build 1.8.0_392-8u392-ga-1~22.04-b08); OpenJDK 64-Bit Server VM (build 25.392-b08, mixed mode)
  Starting server from /home/dummy/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmptij3mx8b
  JVM stdout: /tmp/tmptij3mx8b/h2o_dummy_started_from_python.out
  JVM stderr: /tmp/tmptij3mx8b/h2o_dummy_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_dummy_5c8c99
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.456 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
14:51:55.306: _train param, Dropping bad and constant columns: [pixel729, pixel644, pixel645, pixel448, pixel727, pixel728, pixel560, pixel52, pixel364, pixel760, pixel10, pixel54, pixel167, pixel53, pixel168, pixel56, pixel11, pixel55, pixel643, pixel14, pixel57, pixel16, pixel15, pixel59, pixel18, pixel17, pixel19, pixel754, pixel755, pixel756, pixel757, pixel758, pixel759, pixel83, pixel196, pixel82, pixel85, pixel671, pixel84, pixel111, pixel672, pixel112, pixel673, pixel113, pixel476, pixel753, pixel392, pixel700, pixel701, pixel141, pixel780, pixel30, pixel781, pixel782, pixel32, pixel420, pixel783, pixel31, pixel421, pixel587, pixel33, pixel140, pixel699, pixel139, pixel8, pixel615, pixel9, pixel6, pixel7, pixel4, pixel5, pixel2, pixel3, pixel0, pixel21, pixel1, pixel20, pi

██
14:54:49.668: _train param, Dropping unused columns: [pixel729, pixel644, pixel645, pixel448, pixel727, pixel728, pixel560, pixel52, pixel364, pixel760, pixel10, pixel54, pixel167, pixel53, pixel168, pixel56, pixel11, pixel55, pixel643, pixel14, pixel57, pixel16, pixel15, pixel59, pixel18, pixel17, pixel19, pixel754, pixel755, pixel756, pixel757, pixel758, pixel759, pixel83, pixel196, pixel82, pixel85, pixel671, pixel84, pixel111, pixel672, pixel112, pixel673, pixel113, pixel476, pixel753, pixel392, pixel700, pixel701, pixel141, pixel780, pixel30, pixel781, pixel782, pixel32, pixel420, pixel783, pixel31, pixel421, pixel587, pixel33, pixel140, pixel699, pixel139, pixel8, pixel615, pixel9, pixel6, pixel7, pixel4, pixel5, pixel2, pixel3, pixel0, pixel21, pixel1, pixel20, pixel23, pixel532, pixel730, pixel22, pixel731, pixel25, pixel24, pixel27, pixel26, pixel29, pixel28]

█
14:54:52.510: _train param, Dropping unused columns: [pixel729, pixel644, pixel645, pixel448, pixel727, pixel728,

H2OAutoMLClassifier(max_runtime_secs=180)

In [ ]:
from sklearn.metrics import accuracy_score

Y_pred_h2o = aml.predict(X_test)
h2o_acc = round(accuracy_score(y_test,Y_pred_h2o)*100, 2)
h2o_acc

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


96.65

In [ ]:
Y_pred_h2o = aml.predict(df_test)
submission = pd.DataFrame({
    'ImageId': test['ImageId'],
    'Label' : Y_pred_h2o
})

submission.to_csv("submission_H2O.csv", index = False)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [ ]:
#Auto gluon
from autogluon.tabular import TabularDataset, TabularPredictor

# autogluon은 label 이 존재하는 train data가 필요함
label_str = "label"
save_path = 'autogluon_mnist'
atg_prd = TabularPredictor(label = label_str, path = save_path).fit(df_train)

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "autogluon_mnist/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct 5 21:02:42 UTC 2023
Disk Space Avail:   962.10 GB / 1081.10 GB (89.0%)
Train Data Rows:    42000
Train Data Columns: 784
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	10 unique label values:  [1, 0, 4, 7, 3, 5, 8, 9, 2, 6]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Dat

	6.31s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.97	 = Validation score   (accuracy)
	1110.97s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9584	 = Validation score   (accuracy)
	4.64s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9596	 = Validation score   (accuracy)
	4.1s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: XGBoost ...
	0.9672	 = Validation score   (accuracy)
	115.52s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.9736	 = Validation score   (accuracy)
	68.95s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: LightGBMLarge ...
	0.9712	 = Validation score   (accuracy)
	166.52s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.9832	 = Validation score   (accuracy)
	0.62s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training 

In [ ]:
predictor = TabularPredictor.load(save_path)
Y_pred_atg = predictor.predict(X_test)

In [ ]:
predictor.leaderboard(df_train)

                  model  score_test  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    0.999000     0.9832        8.853370       0.808309    49.940766                 0.008084                0.000604           0.621197            2       True         14
1            LightGBMXT    0.998429     0.9736        1.089136       0.067798    58.989950                 1.089136                0.067798          58.989950            1       True          4
2              LightGBM    0.998381     0.9728        1.155568       0.074401    68.863529                 1.155568                0.074401          68.863529            1       True          5
3         LightGBMLarge    0.998286     0.9712        2.511168       0.142022   166.522720                 2.511168                0.142022         166.522720            1       True         13
4               XGBoost    0.9

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.999000,0.9832,8.853370,0.808309,49.940766,0.008084,0.000604,0.621197,2,True,14
1,LightGBMXT,0.998429,0.9736,1.089136,0.067798,58.989950,1.089136,0.067798,58.989950,1,True,4
2,LightGBM,0.998381,0.9728,1.155568,0.074401,68.863529,1.155568,0.074401,68.863529,1,True,5
3,LightGBMLarge,0.998286,0.9712,2.511168,0.142022,166.522720,2.511168,0.142022,166.522720,1,True,13
4,XGBoost,0.998048,0.9672,1.395633,0.085909,115.519564,1.395633,0.085909,115.519564,1,True,11
5,KNeighborsDist,0.998048,0.9672,6.747214,0.499415,0.188774,6.747214,0.499415,0.188774,1,True,2
6,NeuralNetFastAI,0.997643,0.9804,0.633623,0.034355,40.360998,0.633623,0.034355,40.360998,1,True,3
7,RandomForestGini,0.997595,0.9596,0.661540,0.099639,5.643395,0.661540,0.099639,5.643395,1,True,6
8,ExtraTreesEntr,0.997595,0.9596,0.771801,0.100779,4.099276,0.771801,0.100779,4.099276,1,True,10
9,ExtraTreesGini,0.997524,0.9584,0.653720,0.099868,4.642270,0.653720,0.099868,4.642270,1,True,9


![image.png](attachment:image.png)

In [ ]:
atg_acc = round(accuracy_score(y_test,Y_pred_atg)*100, 2)
atg_acc

99.9

In [ ]:
Y_pred_atg = aml.predict(df_test)
submission = pd.DataFrame({
    'ImageId': test['ImageId'],
    'Label' : Y_pred_atg
})

submission.to_csv("submission_atg.csv", index = False)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [ ]:
%%time
#AutoKeras
import tensorflow as tf
import autokeras as ak
warnings.filterwarnings('ignore')
logging.captureWarnings(True)
simplefilter(action='ignore', category=FutureWarning)

CPU times: user 60 µs, sys: 1 µs, total: 61 µs
Wall time: 68.2 µs


In [ ]:
%%time
auk = ak.StructuredDataClassifier(overwrite = False, max_trials = 10)
auk.fit(X_train, y_train, epochs = 10)

INFO:tensorflow:Reloading Tuner from ./structured_data_classifier/tuner0.json


INFO:tensorflow:Reloading Tuner from ./structured_data_classifier/tuner0.json


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10


2024-01-08 17:02:24.478605: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b58160d000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-08 17:02:24.478644: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-01-08 17:02:24.489700: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-08 17:02:24.519516: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-01-08 17:02:24.599131: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1050/1050 [==============================] - 125s 113ms/step - loss: 0.8517 - accuracy: 0.7408
Epoch 2/10
1050/1050 [==============================] - 140s 133ms/step - loss: 0.5304 - accuracy: 0.8356
Epoch 3/10
1050/1050 [==============================] - 145s 138ms/step - loss: 0.4595 - accuracy: 0.8544
Epoch 4/10
1050/1050 [==============================] - 139s 133ms/step - loss: 0.4121 - accuracy: 0.8702
Epoch 5/10
1050/1050 [==============================] - 131s 125ms/step - loss: 0.3841 - accuracy: 0.8796
Epoch 6/10
1050/1050 [==============================] - 131s 124ms/step - loss: 0.3532 - accuracy: 0.8870
Epoch 7/10
1050/1050 [==============================] - 131s 124ms/step - loss: 0.3248 - accuracy: 0.8925
Epoch 8/10
1050/1050 [==============================] - 131s 125ms/step - loss: 0.3110 - accuracy: 0.8993
Epoch 9/10
1050/1050 [==============================] - 133s 127ms/step - loss: 0.2927 - accuracy: 0.9045
Epoch 10/10
1050/1050 [==============================] - 

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


CPU times: user 47min 30s, sys: 18min 53s, total: 1h 6min 24s
Wall time: 44min 34s


In [ ]:
from sklearn.metrics import accuracy_score
Y_pred_auk = auk.predict(X_test)

Y_pred_auk = Y_pred_auk.flatten().astype(int)

auk_acc = round(accuracy_score(y_test,Y_pred_auk)*100, 2)

KeyboardInterrupt: 

In [ ]:
auk_acc

87.42

In [ ]:
print(y_test)
print('-'*30)
print(Y_pred_auk)

5457     8
38509    1
25536    9
31803    9
39863    8
        ..
8388     4
29359    9
40276    3
18421    0
4335     9
Name: label, Length: 8400, dtype: int64
------------------------------
[8 1 9 ... 3 0 9]


array([8, 1, 9, ..., 3, 0, 9])

In [ ]:
Y_pred_auk = auk.predict(df_test)
Y_pred_auk = Y_pred_auk.flatten().astype(int)
#Y_pred_auk_pd = pd.DataFrame(Y_pred_auk)

# Y_pred_auk 의 값이 1차원 배열값으로 나옴 -> list로 변환


'''
li = []
for i in Y_pred_auk:
    li.append(i[0])
'''

submission = pd.DataFrame({
    'ImageId': test['ImageId'],
    'Label' : Y_pred_auk
})

submission.to_csv("submission_auk.csv", index = False)

875/875 [==============================] - 93s 107ms/step


In [ ]:
dataset = pd.read_csv('./submission_auk.csv')

In [ ]:
dataset

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,4


In [ ]:
import sys
sys.version

'3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'